In [ ]:
import ddm_data_simulation as ddm_sim
import scipy as scp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neighbors import KernelDensity
#import kde_class
import multiprocessing as mp
import psutil
import pickle 
import os
import re

import kde_training_utilities as kde_util
import kde_class as kde

In [ ]:
kde_util.kde_train_test_from_simulations_flexbound(base_simulation_folder = os.getcwd() + '/data_storage/kde/base_simulations/',
                                         target_folder = os.getcwd() + '/data_storage/kde/kde_ddm_flexbound_train_test_extended_params_2/',
                                         n_total = 55000000,
                                         p_train = 0.9,
                                         mixture_p = [0.8, 0.1, 0.1],
                                         model = 'ddm_flexbound',
                                         print_info = False,
                                         target_file_format = 'pickle',
                                         n_files_max = 'all')